## Thiago Pires

:::: columns
::: {.column}
![](avatar.jpeg){.rounded-corners .absolute left=5%}
:::
::: {.column}
<h4>Formação:</h4>
<ul>
<li>Estatística (UERJ)</li>
<li>MSc. Epidemiologia (ENSP/FIOCRUZ)</li>
<li>DSc. Engenharia Biomédica (COPPE/UFRJ)</li>
</ul><br>

<h4>Experiência profissional:</h4>
<ul>
<li>Pesquisador (FIOCRUZ)</li>
<li>Professor (UFF)</li>
<li>Estatístico (FGV)</li>
<li>Cientista de Dados (IBM)</li>
</ul>
:::
::::

# TinyML

> É uma abreviação para *Tiny Machine Learning*. Refere-se a uma abordagem de aplicação de modelos de aprendizado de máquina em dispositivos de recurso computacional limitado, como microcontroladores, microprocessadores e sistemas embarcados.

## Microcontroladores

> É um pequeno computador num único circuito integrado

\

:::: columns
::: {.column}
- Contendo um núcleo de processador
- Memória
- Periféricos programáveis de entrada e saída. 
- WIFI e Bluetooth.
:::

::: {.column}

<center>
![**ESP32** é uma série de microcontroladores de baixo custo e baixo consumo de energia.](esp32.jpeg){width="40%"}
</center>

:::
::::

## ATOM Lite

:::: columns
::: {.column width="40%"}
- G22, G19, etc: General Purpose Input/Output (GPIO) são portas programáveis de entrada e saída de dados.
- I2C, SPI: Protocolos de comunicação.
- GND, 3V3, 5V: Energia. 
:::
::: {.column width="60%"}
<center>
![ATOM Lite da M5Stack](atom.jpeg){width="100%"}
</center>

:::
::::

## Protocolos de comunicação

### I2C

:::: columns
::: {.column}
O protocolo I2C (Inter-Integrated Circuit) é uma das interfaces de comunicação serial mais antigas e amplamente usadas no mundo da eletrônica. Sua história remonta à década de 1980, quando foi desenvolvido pela Philips 
:::
::: {.column}

```{mermaid}
sequenceDiagram
    participant Master as Microcontrolador
    participant Slave as Periférico

    Master->>Slave: Microcontrolador inicia comunicação
    note over Master,Slave: SDA muda de alta para baixa voltagem antes do SCL
    Master->>Slave: Envio do endereço do periférico (7 bits)
    note over Master,Slave: Inclusão R/W bit (0 para escrita)
    Slave->>Master: Acknowledge (ACK)
    note over Slave,Master: Reconhecimento do periférico com um bit
    Master->>Slave: Envio de dados
    note over Master,Slave: 8 bits
    Slave->>Master: Acknowledge (ACK)
    note over Slave,Master: Periférico reconhece
    Master-->>Slave: Condição de parada
    note over Slave,Master: SDA muda de baixa para alta voltagem após o SCL
```

:::
::::

## Protocolos de comunicação

### UART

:::: columns
::: {.column}
O protocolo UART (Universal Asynchronous Receiver/Transmitter)
:::
::: {.column}

```{mermaid}
sequenceDiagram
    participant DispositivoA as Dispositivo A (Transmissor)
    participant DispositivoB as Dispositivo B (Receptor)

    DispositivoA->>DispositivoB: Início da Comunicação UART
    DispositivoA->>DispositivoB: TX para o RX
    note over DispositivoA, DispositivoB: TX de um dispositivo <br> transmite para o RX do outro
    DispositivoA-->>DispositivoB: Fim da Comunicação UART
```

:::
::::

## Linguagens de programação para o ESP32

- C, C++
- Python (Micropython)
- Rust
- Go (TinyGo)
- Javascript (Espruino)

## Lego Technic

:::: columns
::: {.column}
**LEGO Technic** é um tema de produtos LEGO voltada para a criação de modelos mais complexos, com o recurso a blocos e peças mais sofisticados do que os das linhas básica e temáticas de LEGO
:::
::: {.column}
<center>
![Peças do Lego Technic](lego.jpeg){width="45%"}
</center>

:::
::::

## Triciclo Reverso

![Triciclo reverso de lego](){width="45%"}

## Controle remoto via socket

![Esquema do projeto]()

## Controle remoto via socket

### Micropython


In [ ]:
#| eval: false

import machine
import struct
import network
import socket
import neopixel
import utime

# setup uart
uart = machine.UART(1, tx=19, rx=22)
uart.init(115200, bits=8, parity=None, stop=1)

# setup led
np = neopixel.NeoPixel(machine.Pin(27), 1)

# setup servo and motor
sda_pin = machine.Pin(25)
scl_pin = machine.Pin(21)

i2c = machine.I2C(0, sda=sda_pin, scl=scl_pin, freq=400000)
devices = i2c.scan()

device = [i for i in devices if hex(i) == '0x38'][0]

def set_speed(speed):
    buf = bytearray(1)
    struct.pack_into('b', buf, 0, speed)
    i2c.writeto_mem(device, 0, buf)

def set_angle(angle):
    buf = bytearray(1)
    struct.pack_into('b', buf, 0, angle)
    i2c.writeto_mem(device, 2, buf)

def set_direction(x):
    if x > 200:
        set_angle(65)
    elif x < 50:
        set_angle(115)
    else:
        set_angle(90)
        
def set_run(x):
    if x > 200:
        set_speed(127)
    elif x < 50:
        set_speed(0)
    else:
        set_speed(86)

# create access-point
ap = network.WLAN(network.AP_IF)
ap.config(essid='ATOM-MOTION')
ap.config(max_clients=10)
ap.active(True)

# create server socket
server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server.bind(('', 12000))

while True:
    
    try:
        status = int(uart.read(1).decode())
        if status == 1:
            np[0] = (0,0,255)
            np.write()
        elif status == 0:
            np[0] = (0, 255, 0)
            np.write()
            direction, address_client = server.recvfrom(2048)
            out = struct.unpack('BBB', direction)
            set_direction(out[0])
            set_run(out[1])
            print(out)
        else:
            np[0] = (255, 0, 0)
            np.write()
        print(status)
    except:
        pass
    
    utime.sleep_ms(500)

## Camera para visão computacional

## Controle por visão computacional

## 






































##

:::: columns
::: {.column}
::: {.medium right="70%"} 

<h1>Obrigado</h1>

:::
:::
::: {.column}
::: {.medium right="30%"}

[th1460.github.io](https://th1460.github.io/)<br>
[github.com/th1460](https://github.com/)<br>
[medium.com/@thopr](https://medium.com/@thopr)<br>
[linkedin.com/in/thop](https://www.linkedin.com/in/thop)<br>
[twitter.com/th14600](https://twitter.com/th14600)<br>
[@thop@fosstodon.org](https://fosstodon.org/@thop)

:::
:::
::::